## RNN Recurrent Neural Network
卷曲神经网络
- Embedding 与变长输入的处理
- 序列式问题
- 循环神经网络
- LSTM模型,拥有更好的记忆能力

长短期记忆（Long short-term memory, LSTM）是一种特殊的RNN，主要是为了解决长序列训练过程中的梯度消失和梯度爆炸问题。简单来说，就是相比普通的RNN，LSTM能够在更长的序列中有更好的表现。


## 为什么需要循环神经网络
合并＋padding的缺点
- 信息丢失
    - 多个embedding合并，pad变成噪音，就算没有许多pad，也会导致数据无主次
    - 有些无关词语过多会导致数据被稀释掉
- 无效计算太多，抵消，太多的pad噪音，和次要词都会进行计算，务必要 

## 序列式问题
- 普通神经网络：`x-rnn-y`
- 图片生成描述：`x-rnn-[y1,y2,...]`
- 文本分类（文本情感分析）：`[x1,x2,...]-rnn-y`
- 多对多，encoding-decoding 机器翻译：`[x1,x2,...]-rnn-[y1,y2,...]`

## 循环神经网络
- 维护一个状态作为下一步的额外输入
- 每一步使用同样的激活函数和参数
- 每一步不仅得到一个输出，还得到一个状态数值

公式,最简单的循环神经网络：
$$s_t = f_w(s_{t-1},x_t) ,s_t 新状态 s_{t-1} 为旧状态 $$
将以上公式展开：
$$s_t = tanh(W_{s_{t-1}},u_{x_t}),W_{s_{t-1}},u_{x_t},分别是对s_{t-1},x_t数据的矩阵变换$$
$$\hat{y_t} = softmax(V_{s_t}),相当于对s_t做一个全连接层$$

### 循环神经网络做文本分类
```
x1 - rnn1
x2 - rnn2
x3 - rnn3   ===合并==mlp==y
x4 - rnn4
x5 - rnn5

奇中x1-x5输入到embedding层的rnn层后会产生一个和上下rnn层相互作用的状态值
```

In [23]:
import os, sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf

from tensorflow import keras


print(sys.version_info)
for module in tf, mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)



sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
tensorflow 2.1.0
matplotlib 3.2.0
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [24]:
# 使用数据集imdb  
imdb = keras.datasets.imdb
# 设定词表的个数，统计所有的词语并将前一万个保留下来
vocab_size = 10000
# 词表的index从3开始算
index_from =3
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size, index_from=index_from)

In [ ]:
print(train_data[0], train_labels[0])
print(train_data.shape, train_labels.shape)
print(len(train_data[0]), len(train_data[1]))

In [ ]:
print(test_data.shape, test_labels.shape)

In [ ]:
word_index = imdb.get_word_index()
print(len(word_index))
print(word_index)

In [ ]:
# 将整体的序列向右移三位，如下添加四个自定义的字符
word_index = {k:(v+3) for k,v in word_index.items()}

In [ ]:
# padding填充的字符
word_index['<PAD>'] = 0 
# 每个字符开始的字符
word_index['<START>'] = 1
# 不知道的字符设置
word_index['<UNK>'] = 2
# 每个字符结束的字符
word_index['<END>'] = 3

# 将word_index 反转成index：word
reverse_word_index = {v:k for k, v in word_index.items()}

decode_review = lambda test_ids:''.join([reverse_word_index.get(word_id, '<UNK>') for word_id in test_ids])

decode_review(train_data[0])

In [ ]:
max_length = 500

train_data = keras.preprocessing.sequence.pad_sequences(
    train_data, # list of list
    value=word_index['<PAD>'],
    padding='post', # post放在补全的后面，pre放在补全的前面
    maxlen = max_length,
)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data, # list of list
    value=word_index['<PAD>'],
    padding='post', # post放在补全的后面，pre放在补全的前面
    maxlen = max_length,
)

print(train_data[0])

In [ ]:
# 每个embedding定为长度为16的向量
max_length = 500
embedding_dim = 16
batch_size = 128
# 实现一个单层单向的LSTM
bi_rnn_model = keras.models.Sequential([
    # 1. define matrix : [vocab_size, embedding_dim]
    # 2. [1,2,3,4..], max_length * embedding_dim
    # 3. batch_size * max_length * embedding_dim
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # 添加一个rnn层，先定义一个简单的rnn网络层，return_sequences= False表示我们只是用最后的输出
    keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=False)),
    # 全连接层
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'),

])
bi_rnn_model.summary()
bi_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 0.2作为验证集
bi_rnn_history = bi_rnn_model.fit(train_data, train_labels, epochs=30, batch_size=batch_size, validation_split=0.2)

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()



In [ ]:
plot_learning_curves(single_rnn_history, 'accuracy', 30, 0, 1)
plot_learning_curves(single_rnn_history, 'loss', 30, 0, 3)

In [ ]:
bi_rnn_model.evaluate(test_data, test_labels, batch_size=batch_size)

## 图像分析
过拟合现象
对valid数据loss比较小，但是对于预测数据loss过于大
解决办法：
过拟合解决方法：

1. 增加训练数据，数据量变多
2. L1、L2...regularization
    - y=Wx
    - l1 : cost = (Wx-realy)^2 + abs(W)
    - l2 : cost = (Wx-realy)^2 + sqrt(W)
    - l3 : cost = (Wx-realy)^2 + pow(W)
3. 简化神经网络结构
    - droput regularization

跑一次训练集要好久，顶不住，方法mark了以后加入到自己的项目做分析。。